<p>
<img src="../imgs/EII-ULPGC-logo.jpeg" width="430px" align="right">

# **NOTEBOOK 13**
---

# **Modelos del lenguaje basados en redes neuronales artificiales**

## **Redes neuronales recurrentes (RNN)**

### **Ejemplo de aplicación de un modelo LSTM a la clasificación de textos**

#### **Dataset**

El conjunto de datos "AG_NEWS" es un conjunto de datos de clasificación de texto ampliamente utilizado en el campo del procesamiento de lenguaje natural (NLP). Contiene noticias de diferentes categorías y se utiliza comúnmente para tareas de clasificación de texto. El conjunto de datos AG_NEWS consta de noticias de cuatro categorías principales, que son:

1. **World**: Noticias sobre eventos y acontecimientos globales, como política internacional, relaciones internacionales y noticias mundiales en general.

2. **Sports**: Noticias relacionadas con eventos deportivos, resultados de partidos, eventos deportivos nacionales e internacionales, etc.

3. **Business**: Noticias relacionadas con el mundo de los negocios, finanzas, economía, empresas, informes de ganancias y otros temas económicos.

4. **Sci/Tech**: Noticias relacionadas con ciencia y tecnología, incluyendo avances científicos, novedades tecnológicas, gadgets, investigaciones científicas y más.

Cada instancia del conjunto de datos AG_NEWS generalmente consiste en un título y un cuerpo de una noticia, junto con una etiqueta que indica la categoría a la que pertenece.

In [ ]:
!pip uninstall torch torchtext -y
!pip install torch==2.0.1 torchtext==0.15.2 --index-url https://download.pytorch.org/whl/cu118
!pip install portalocker>=2.0.0

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 453.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89991 sha256=8f8c790232a83d04fd4578f743579fcbef00f73400f3838e73aaa039a9214188
  Stored in directory: /root/.cache/pip/wheels/27/2c/b6/3ed2983b1b44fe0dea1bb35234b09f2c22fb8ebb308679c922
Successfully built lit
ERROR: pip's dependency resolver does not currently take into account all the packages that are ins

In [ ]:
from torchtext import datasets
from torchtext.data import to_map_style_dataset
import numpy as np

# Load the dataset
train_iter, test_iter = datasets.AG_NEWS(split=('train', 'test'))

train_ds = to_map_style_dataset(train_iter)
test_ds = to_map_style_dataset(test_iter)

train = np.array(train_ds)
test = np.array(test_ds)

#### **Tokenización**

In [ ]:
# Create vocabulary and embedding

from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer("basic_english")

vocab = build_vocab_from_iterator(map(lambda x: tokenizer(x[1]), train_iter), specials=['<pad>','<unk>'])
vocab.set_default_index(vocab["<unk>"])

In [ ]:
print("Tamaño del vocabulario:", len(vocab), "tokens")
print("Tokenización de la frase 'Here is an example sentence':", tokenizer("Here is an example sentence"))
print("Índices de las palabras 'here', 'is', 'an', 'example', 'supercalifragilisticexpialidocious':", vocab(['here', 'is', 'an', 'example', 'supercalifragilisticexpialidocious']))
print("Palabras correspondientes a los índices 475, 21, 30, 5297, 0:", vocab.lookup_tokens([475, 21, 30, 5297, 0]))
print("Las diez primeras palabras del vocabulario:", vocab.get_itos()[:10])

Tamaño del vocabulario: 95812 tokens
Tokenización de la frase 'Here is an example sentence': ['here', 'is', 'an', 'example', 'sentence']
Índices de las palabras 'here', 'is', 'an', 'example', 'supercalifragilisticexpialidocious': [476, 22, 31, 5298, 1]
Palabras correspondientes a los índices 475, 21, 30, 5297, 0: ['version', 'at', 'from', 'establish', '<pad>']
Las diez primeras palabras del vocabulario: ['<pad>', '<unk>', '.', 'the', ',', 'to', 'a', 'of', 'in', 'and']


In [ ]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

print("Tokenización de la frase 'Here is an example sentence':", text_pipeline("Here is an example sentence"))

Tokenización de la frase 'Here is an example sentence': [476, 22, 31, 5298, 2994]


#### **DataLoader**

En PyTorch, **DataLoader** se utiliza para cargar y manejar datos de manera eficiente durante el entrenamiento de modelos de aprendizaje profundo, especialmente en tareas de aprendizaje supervisado como la clasificación, la regresión y más. El DataLoader forma parte de la biblioteca torch.utils.data, y su objetivo principal es facilitar la administración de lotes (batches) de datos y la distribución de esos lotes al modelo de forma automática.

La función <code>collate_batch</code> es una función personalizada que se utiliza como argumento para collate_fn al crear instancias de los objetos DataLoader. Tiene la responsabilidad de procesar y agrupar las muestras individuales de datos dentro de un lote (batch) de manera que sean compatibles para su posterior procesamiento dentro de la LSTM. Es especialmente útil cuando las secuencias de texto tienen longitudes diferentes y es necesario realizar un relleno (padding) para que todas tengan la misma longitud.

In [ ]:
from torch.utils.data import DataLoader
import torch


def collate_batch(batch):
    label_list, text_list = [], []
    for sample in batch:
        label, text = sample
        text_list.append(torch.tensor(text_pipeline(text), dtype=torch.long))
        label_list.append(label_pipeline(label))
    return torch.tensor(label_list, dtype=torch.long), torch.nn.utils.rnn.pad_sequence(text_list, batch_first=True, padding_value=vocab["<pad>"])

train_dataloader = DataLoader(
    train_iter, batch_size=64, shuffle=True, collate_fn=collate_batch
)

test_dataloader = DataLoader(
    test_iter, batch_size=64, shuffle=True, collate_fn=collate_batch
)

Para verificar que estamos creando correctamente los lotes, vamos a imprimir las primeras cuatro instancias:

In [ ]:
for batch in train_dataloader:
    print(batch[1][:4])
    print("\n")
    print(batch[0][:4])
    print("\n")
    break

tensor([[  640,   431, 16431,     7,   222,  3295,   576,     7,   337,   326,
           114,    91,    50,    36,     6,   640, 20317,    21, 14357,     4,
            50,   704,    44,    12,  1490,    62,   133,   178,    20,     6,
          4115,  2664,  4700,     6,    24,  5726,  1032,  7885,    19,    31,
         21040,  7328,    18,  2376,  5970,   480,     8,     3, 21097,     2,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0],
        [ 7645,    51,    17,   743,  6725,   712,  7391,    17,   103,    34,
           799,     5,  6725,    47, 15204,     9,   712,  7391,   303,     7,
         10694,  1067,  1310,     4,   454,   234,     2,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0, 

#### **Modelo LSTM**

Definimos una clase llamada `LSTMTextClassificationModel` para la clasificación de texto basado en LSTM. Esta clase hereda de `nn.Module`, la clase base para todos los modelos en PyTorch.

**Inicialización del Modelo**: En el método `__init__`, se definen los componentes principales del modelo y se configuran sus parámetros:
   - **vocab_size**: El tamaño del vocabulario, es decir, la cantidad de palabras únicas en el conjunto de datos de entrenamiento.
   - **embed_dim**: La dimensión de los vectores de embedding para representar las palabras.
   - **hidden_dim**: El tamaño de la capa oculta de la LSTM, que controla la cantidad de unidades o "memoria" en la red LSTM.
   - **num_class**: El número de clases de salida en la tarea de clasificación.


**Capa de Embedding**: Se define una capa de embedding (`nn.Embedding`) que se utilizará para representar las palabras como vectores densos. Esta capa no es pre-entrenada, lo que significa que los vectores de embedding se entrenarán junto con el modelo durante el proceso de entrenamiento.

**Capa LSTM**: Se define una capa LSTM (`nn.LSTM`) que toma los vectores de embedding como entrada. `embed_dim` es la dimensión de entrada de la capa LSTM, y `hidden_dim` es la dimensión de su capa oculta. `batch_first=True` indica que los datos de entrada tendrán la forma `(batch_size, sequence_length, embed_dim)`.

**Capa Fully Connected (FC)**: Se define una capa completamente conectada (`nn.Linear`) que se utiliza para producir las salidas de clasificación. Toma las salidas de la capa LSTM correspondientes a la última iteración y las reduce a `num_class` dimensiones, que corresponde al número de clases en la tarea de clasificación.


In [ ]:
import torch
import torch.nn as nn

class LSTMTextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super(LSTMTextClassificationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)  # <-- Capa de embedding genérica (no pre-entrenada)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_class)

    def forward(self, text):
        embedded = self.embedding(text)  # <-- Tras pasar por la capa de embedding, las palabras se representan como vectores
        lstm_out, _ = self.lstm(embedded)
        # Tomar la última salida de la secuencia LSTM
        last_output = lstm_out[:, -1, :]
        output = self.fc(last_output)
        return output


Antes de seguir veamos cómo se conforma un *batch* de datos tras pasar la capa de *embedding*. Como se muestra en la figura siguiente, un *batch* es un vector de tres dimensiones, donde la primera dimensión es el tamaño del *batch*, la segunda dimensión es el número de palabras en cada secuencia y la tercera dimensión es el tamaño del vector de *embedding* (*channels* o *features*).

<img src="imgs/Lote1_b.svg" width="30%">

Por ejemplo, un *token* estaría ahora representado por un vector de *embedding* de tres componentes, en lugar de un escalar. Si lo quisiéramos referenciar sería: <code>mini_lote[0,3,:]</code>.

<img src="imgs/Lote2_b.svg" width="30%">


Fíjate que la tercera dimensión del tensor de salida de la LSTM no tiene por qué tener el mismo tamaño que la tercera dimensión del tensor de entrada. La parte del tensor correspondiente a la línea de código <code>last_output = lstm_out[:, -1, :]</code> sería parecida a la siguiente:

<img src="imgs/Lote3_b.svg" width="30%">

Ya podemos pasar un primer mini-lote de datos por la red. Para ello, vamos a crearlo a partir de los datos de entrenamiento. Esto solo lo hacemos para asegurarnos de que el código funciona correctamente.

In [ ]:
model = LSTMTextClassificationModel(len(vocab), 32, 64, 4)
model.train()

for batch in train_dataloader:
    predicted_label = model(batch[1])
    label = batch[0]
    break

print(batch[1][:4])
print(predicted_label[:4])
print(label[:4])

tensor([[    3,   270,  1683,  1668,     3,  3648, 37474,  7404,  1086,   391,
          1828,    17,    10,   667,    43,   278,  8230,    25,  1879,   353,
             7,     3,  3127,     4,  1291,     9,  2754,  3142,     2,    25,
          1163, 87558,    30,  4608,  2109,     2,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/iter/combining.py:297: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


#### **Entrenamiento**

Vamos a construir las funciones <code>train</code> y <code>evaluate</code>. Dentro de la función <code>evaluate</code> vamos a prestar atención al contexto <code>with torch.no_grad():</code>, que sirve para indicar que no se van a calcular los gradientes. Esto es así porque en la fase de evaluación no se van a actualizar los pesos de la red, solo se van a utilizar para calcular la precisión de la red. Esto optimiza los recursos utilizados por PyTorch en cuanto a la gestión de la memoria.

Recuerda que en PyTorch, `model.train()` y `model.eval()` son métodos que se utilizan para cambiar el modo de entrenamiento de un modelo de aprendizaje profundo. Estos métodos afectan el comportamiento de ciertos módulos en el modelo, como las capas de dropout y normalización, que se comportan de manera diferente durante el entrenamiento y la evaluación. Específicamente hacen lo siguiente:

**model.train()**:
   - Cuando se llama a `model.train()`, el modelo se coloca en modo de entrenamiento.
   - En este modo, las capas de dropout se activan, lo que significa que se aplican durante el entrenamiento para ayudar a evitar el sobreajuste. Durante el entrenamiento, las capas de dropout "apagan" aleatoriamente un porcentaje de las unidades (neuronas) en la red en cada paso de entrenamiento.
   - También afecta a las capas de normalización, como Batch Normalization, para que utilicen estadísticas de mini-lotes (batches) durante el entrenamiento.

**model.eval()**:
   - Cuando se llama a `model.eval()`, el modelo se coloca en modo de evaluación o inferencia.
   - En este modo, las capas de dropout se desactivan, lo que significa que no se aplican durante la evaluación. Esto garantiza que el modelo produzca resultados deterministas y coherentes durante la inferencia.
   - Las capas de normalización, como Batch Normalization, utilizan estadísticas acumuladas durante el entrenamiento (en lugar de estadísticas de mini-lotes) para garantizar una evaluación coherente y precisa.


En la práctica, es común utilizar `model.train()` antes de cada época de entrenamiento y `model.eval()` antes de realizar inferencia o evaluación en un modelo entrenado. Cambiar entre estos modos es esencial para garantizar que el modelo se comporte correctamente en diferentes etapas del proceso de entrenamiento y evaluación.

In [ ]:
import time

# Hyperparameters
EPOCHS = 10  # epoch
LR = 5  # learning rate

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

def train(dataloader):
    model.train()
    total_acc, total_count, max_acc = 0, 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| {:5d} batches '
                  '| accuracy {:8.3f}'.format(idx, total_acc / total_count))

            if max_acc < total_acc / total_count:
                max_acc = total_acc / total_count

            total_acc, total_count = 0, 0
            start_time = time.time()
    return max_acc


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [ ]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()

    accu_train = train(train_dataloader)
    accu_val = evaluate(test_dataloader)

    #if accu_train > accu_val:
    #    scheduler.step()

    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

|   500 batches | accuracy    0.255
|  1000 batches | accuracy    0.252
|  1500 batches | accuracy    0.255
-----------------------------------------------------------
| end of epoch   1 | time: 173.12s | valid accuracy    0.251 
-----------------------------------------------------------
|   500 batches | accuracy    0.262


KeyboardInterrupt: 

---

## **Práctica 1: Uso de *embeddings* preentrenados**

Modifica el código anterior para adaptar el modelo LSTM al uso de *embeddings* preentrenados. Para ello, usa <code>from torchtext.vocab import GloVe</code> y elige el conjunto de *embeddings* GloVe que prefieras. Puedes encontrar más información en https://pytorch.org/text/stable/vocab.html#torchtext.vocab.GloVe

Verifica si se produce una mejora en la precisión del modelo. ¿Qué ocurre si usas un conjunto de *embeddings* preentrenados de diferentes tamaños?



---

El uso de embeddings preentrenados de mayor dimensión mejora la precisión del modelo levemente, aunque el impacto es más significativo al pasar de dimensiones más bajas que entre dimensiones altas. Además, emplear modelos GloVe más grandes, como el 42B o el 840B, puede proporcionar embeddings más contextualizados debido al mayor volumen de datos en los que han sido entrenados, sin embargo dependiendo de la tarea en la que se vaya a utilizar puede ser demasiado, pudiendo haber modelos preentrenados más pequeños que funcionen correctamente sin la necesidad de modelos grandes que aumenten el coste computacional.

In [ ]:
from torchtext import datasets
from torchtext.data import to_map_style_dataset
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
import torch
import time

# Cargar el conjunto de datos AG_NEWS
train_iter, test_iter = datasets.AG_NEWS(split=('train', 'test'))
train_ds = to_map_style_dataset(train_iter)
test_ds = to_map_style_dataset(test_iter)

# Tokenizador y vocabulario
tokenizer = get_tokenizer("basic_english")
vocab = build_vocab_from_iterator(
    map(lambda x: tokenizer(x[1]), train_iter), specials=['<pad>', '<unk>']
)
vocab.set_default_index(vocab["<unk>"])


In [ ]:
from torchtext.vocab import GloVe

# Crear y almacenar embeddings GloVe
def create_and_store_glove_embeddings(vocab):
    glove_embeddings = {}
    for dim in [50, 100, 200, 300]:
        print(f"Cargando GloVe de dimensión {dim}...")
        glove = GloVe(name="6B", dim=dim)
        pretrained_embeddings = torch.zeros((len(vocab), glove.dim))
        for idx, word in enumerate(vocab.get_itos()):
            if word in glove.stoi:
                pretrained_embeddings[idx] = glove[word]
            else:
                pretrained_embeddings[idx] = torch.randn(glove.dim)
        glove_embeddings[dim] = pretrained_embeddings
    print("Embeddings GloVe generados y almacenados.")
    return glove_embeddings

# Almacenar los embeddings GloVe
glove_embeddings = create_and_store_glove_embeddings(vocab)


Cargando GloVe de dimensión 50...


.vector_cache/glove.6B.zip: 862MB [02:38, 5.43MB/s]                           
100%|█████████▉| 399999/400000 [00:13<00:00, 30701.81it/s]


Cargando GloVe de dimensión 100...


100%|█████████▉| 399999/400000 [00:20<00:00, 19508.09it/s]


Cargando GloVe de dimensión 200...


100%|█████████▉| 399999/400000 [00:35<00:00, 11174.12it/s]


Cargando GloVe de dimensión 300...


100%|█████████▉| 399999/400000 [00:52<00:00, 7631.96it/s]


Embeddings GloVe generados y almacenados.


In [ ]:
import torch.nn as nn

# Modelo LSTM para clasificación de texto
class LSTMTextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class, pretrained_embeddings):
        super(LSTMTextClassificationModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=True)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_class)

    def forward(self, text):
        embedded = self.embedding(text)
        lstm_out, _ = self.lstm(embedded)
        last_output = lstm_out[:, -1, :]
        output = self.fc(last_output)
        return output


In [ ]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preparar DataLoader
def collate_batch(batch):
    labels, texts = zip(*batch)
    labels = torch.tensor([int(label) - 1 for label in labels], dtype=torch.long).to(device)
    texts = [torch.tensor(vocab(tokenizer(text)), dtype=torch.long).to(device) for text in texts]
    texts = pad_sequence(texts, batch_first=True, padding_value=vocab["<pad>"])
    return labels, texts

train_dataloader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_ds, batch_size=32, collate_fn=collate_batch)


In [ ]:
# Configurar el modelo con embeddings de una dimensión específica
def configure_model(dim, glove_embeddings):
    pretrained_embeddings = glove_embeddings[dim]
    model = LSTMTextClassificationModel(
        vocab_size=len(vocab),
        embed_dim=dim,
        hidden_dim=128,
        num_class=4,
        pretrained_embeddings=pretrained_embeddings,
    ).to(device)
    return model


In [ ]:
def train(dataloader, model, optimizer):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)

        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| {:5d} batches '
                  '| accuracy {:8.3f}'.format(idx, total_acc / total_count))
            start_time = time.time()

    return total_acc / total_count

def evaluate(dataloader, model):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)

    return total_acc / total_count


In [ ]:
# Hiperparámetros
EPOCHS = 10
LR = 5
criterion = torch.nn.CrossEntropyLoss()
results = []

# Entrenar con una dimensión específica
def train_with_dimension(dim):
    print(f"\nEntrenando con GloVe de dimensión {dim}...")
    model = configure_model(dim, glove_embeddings)
    optimizer = torch.optim.SGD(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

    # Entrenamiento con medición de tiempo
    for epoch in range(1, EPOCHS + 1):
        epoch_start_time = time.time()

        accu_train = train(train_dataloader, model, optimizer)
        accu_val = evaluate(test_dataloader, model)

        scheduler.step()

        print("-" * 59)
        print(
            "| end of epoch {:3d} | time: {:5.2f}s | "
            "Train Acc: {:8.3f} | Valid Acc: {:8.3f}".format(
                epoch, time.time() - epoch_start_time, accu_train, accu_val
            )
        )
        print("-" * 59)

        # Almacenar resultados finales de esta dimensión
        if epoch == EPOCHS:  # Solo guardar el resultado al final de la última época
            results.append({
                "dimension": dim,
                "train_accuracy": accu_train,
                "valid_accuracy": accu_val,
                "time_per_epoch": time.time() - epoch_start_time
            })


In [ ]:
# Llamar al entrenamiento con diferentes dimensiones
train_with_dimension(50)
train_with_dimension(100)
train_with_dimension(200)
train_with_dimension(300)



Entrenando con GloVe de dimensión 50...
|   500 batches | accuracy    0.252
|  1000 batches | accuracy    0.274
|  1500 batches | accuracy    0.406
|  2000 batches | accuracy    0.493
|  2500 batches | accuracy    0.566
|  3000 batches | accuracy    0.615
|  3500 batches | accuracy    0.652
-----------------------------------------------------------
| end of epoch   1 | time: 21.67s | Train Acc:    0.667 | Valid Acc:    0.866
-----------------------------------------------------------
|   500 batches | accuracy    0.883
|  1000 batches | accuracy    0.888
|  1500 batches | accuracy    0.888
|  2000 batches | accuracy    0.889
|  2500 batches | accuracy    0.889
|  3000 batches | accuracy    0.889
|  3500 batches | accuracy    0.890
-----------------------------------------------------------
| end of epoch   2 | time: 21.56s | Train Acc:    0.890 | Valid Acc:    0.888
-----------------------------------------------------------
|   500 batches | accuracy    0.893
|  1000 batches | accur

In [ ]:
# Imprimir los resultados finales
print("\nResultados comparativos:")
for result in results:
    print(
        "Dimensión: {:3d} | Train Acc: {:8.3f} | Valid Acc: {:8.3f} | Tiempo por última época: {:5.2f}s".format(
            result["dimension"], result["train_accuracy"], result["valid_accuracy"], result["time_per_epoch"]
        )
    )


Resultados comparativos:
Dimensión:  50 | Train Acc:    0.896 | Valid Acc:    0.893 | Tiempo por última época: 21.53s
Dimensión: 100 | Train Acc:    0.909 | Valid Acc:    0.899 | Tiempo por última época: 21.44s
Dimensión: 200 | Train Acc:    0.917 | Valid Acc:    0.909 | Tiempo por última época: 21.06s
Dimensión: 300 | Train Acc:    0.922 | Valid Acc:    0.912 | Tiempo por última época: 21.14s
